In [ ]:
#Evaluating a Prompt using Majority Vote Strategy 

In [1]:
import os, yaml, re
from openai import OpenAI, ChatCompletion
import together

In [2]:
#for accessing together/ai API
with open('credentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)
    
os.environ['TOGETHER_AI_API'] = credentials['TOGETHER_AI_API']
together.api_key = os.environ["TOGETHER_AI_API"]

In [ ]:
import os  
import base64
from openai import AzureOpenAI  
from langchain.chat_models import ChatOpenAI, AzureChatOpenAI

endpoint = os.getenv("#")  
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")  
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "#")  

# Initialize Azure OpenAI Service client with key-based authentication    
client = AzureOpenAI(  
    azure_endpoint=endpoint,  
    api_key=subscription_key,  
    api_version = "2024-12-01-preview"
)

    
# Generate the completion  
def chat_completion(USER_MSG):
    completion = client.chat.completions.create(  
        model=deployment,
        messages=[
    {
        "role": "system",
        "content": [
            {
                "type": "text",
                "text": "You are a helpful assitant to identify the tag for sense of a word"
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": USER_MSG
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": ""
            }
        ]
    }
] ,
        max_tokens=500,  
        temperature=0.0,
        top_p=0.95,  
        frequency_penalty=0,  
        presence_penalty=0,
        stop=None,  
        stream=False
        )
    response_content = completion.choices[0].message.content
    return response_content

# Generate the completion

In [66]:
with open('credentials.yaml') as f:
    cadentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = cadentials['OPENAI_API_KEY']

In [67]:
client = OpenAI() #for OpenAI API
from together import Together #for Together AI API

In [68]:
def complete_model(USER_MESSAGE):
    output = together.Complete.create(
                                    prompt= USER_MESSAGE,
                                    model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
                                    max_tokens=1200,
                                    temperature=0,
                                    
                                    
                                    )
    text = output['output']['choices'][0]['text']
    return text

In [9]:
def complete_model(USER_MESSAGE):
    response = client.chat.completions.create(
                                            model = 'gpt-4o',
                                            messages = [
                                                        {"role": "system", "content" : "You are a helpful assitant to identify the tag for sense of a word"},
                                                        {"role": "user", "content": USER_MESSAGE}              
                                                        ],
                                            temperature=0,
                                            max_tokens=1500
                                            )
    return str(response.choices[0].message.content)

In [4]:
# Read data from file
list_of_lists = []
with open('wordnet.txt', 'r',encoding="utf8") as file:
    for element in file:
        items=element.strip("\n").split("	")
        list_of_lists.append(items)
    

In [6]:
#converting the word into the base word as the sense are kept in base
def word_base(word):
    try:
        word=stemmer.stem(word)
        base_word = lemmatizer.lemmatize(word)
        return base_word
    except:
        return word   
    

In [7]:
#function to retrieve the word meaning from the list_of_list list
#this function will specifically read the sense id and the meaning(gloss) which is required for the processing.
def retrieve_meanings(word, data):
    meanings_dict = {}
    for entry in data:
        if word == entry[1].split(".")[0]:
            if entry[-1] != 'None':
                meanings_dict[entry[1]] = entry[2]+", synonyms :"+entry[-1]+"\n"
            else:
                meanings_dict[entry[1]] = entry[2]+"\n"
    return meanings_dict

In [8]:
meanings = retrieve_meanings("use", list_of_lists)
print(meanings)

{'use.n.04:00': "the act of using, synonyms :['usage', 'utilization', 'utilisation', 'employment', 'exercise']\n", 'use.n.04:01': "(psychology) an automatic pattern of behavior in reaction to a specific situation; may be inherited or acquired through frequent repetition, synonyms :['habit']\n", 'use.n.04:02': "exerting shrewd or devious influence especially for one's own advantage, synonyms :['manipulation']\n", 'use.n.07:00': "what something is used for, synonyms :['function', 'purpose', 'role']\n", 'use.n.07:02': 'a particular service, synonyms :[]\n', 'use.n.07:03': "(law) the exercise of the legal right to enjoy the benefits of owning property, synonyms :['enjoyment']\n", 'use.n.22:00': "(economics) the utilization of economic goods to satisfy needs or in manufacturing, synonyms :['consumption', 'economic_consumption', 'usance', 'use_of_goods_and_services']\n", 'use.v.34:00': "use up, consume fully, synonyms :['expend']\n", 'use.v.34:01': "put into service; make work or employ for 

In [9]:
#Extract the sentence without <wsd> token and the index
def extract_word_and_index(sentence):
    # Find the start and end index of the <WSD> tags
    start_index = sentence.find('<WSD>')    
    end_index = sentence.find('</WSD>')
    

    if start_index != -1 and end_index != -1:
        # Extract the word between <WSD> tags
        word = sentence[start_index + len('<WSD>'):end_index]

        # Remove <WSD> and </WSD> tags from the sentence
        cleaned_sentence = sentence[:start_index] + word+" " + sentence[end_index + len('</WSD>'):]
            
        return cleaned_sentence.strip()

In [10]:
#Function to identify the WSD word from the given sentence and return the WSD word on a sentence
def wsdword(text):
    match = re.search(r'<WSD>(.*?)</WSD>', text)
    if match:
        word_inside_wsd = match.group(1)
        return word_inside_wsd

In [11]:
#Funtion to counts the number of tokens : input and output tokens
from nltk.tokenize import word_tokenize

def count_tokens(text):
    # Tokenize the input text
    tokens = word_tokenize(text)
    
    # Count the tokens in the input
    token_count = len(tokens)
    
    return token_count

In [12]:
#Word lemmatizer to get the base word of the WSD word
import nltk
from nltk.stem import WordNetLemmatizer, PorterStemmer

nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

[nltk_data] Downloading package wordnet to C:\Users\Deshan
[nltk_data]     Sumanathilaka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [13]:
import json
def load_dictionary_from_file(filename):
    try:
        with open(filename, 'r') as file:
            data_dict = json.load(file)
        return data_dict
    except FileNotFoundError:
        # Handle the case where the file doesn't exist
        return None
    except json.JSONDecodeError:
        # Handle the case where the file contains invalid JSON
        return None

In [14]:
def get_pos(input_char):
    pos_mapping = {
        'n': 'NOUN',
        'v': 'VERB',
        'a': 'ADJ',
        'r': 'ADV',
        's': 'SADV'
    }
    
    return pos_mapping.get(input_char.lower(), 'Invalid input')

In [15]:
loaded_dict = load_dictionary_from_file("my_dictionary.json")

In [23]:
#low Freequency Sampling
import random
from collections import defaultdict

def get_balanced_examples_low(input_list):
    # Parse the input list into a dictionary with senseid as key
    senseid_dict = defaultdict(list)
    
    for item in input_list:
        #print(item)
        try:
            example, senseid = item.rsplit(' . ', 1)
            #print(senseid.strip())
            senseid_dict[senseid.strip()].append(item.strip())  # Store the full item (example + senseid)
        except:
            continue
    #Find the minimum number of examples for any senseid
    min_examples = min(len(examples) for examples in senseid_dict.values())
    
    #Randomly sample min_examples from each senseid
    balanced_list = []
    for senseid, examples in senseid_dict.items():
        balanced_list.extend(random.sample(examples, min_examples))
    
    return balanced_list

In [ ]:
examples = examples=loaded_dict["bank"]["NOUN"]
print(len(examples))
examples = get_balanced_Average_examples(examples)
print(len(examples))

In [36]:
import random
from collections import defaultdict

def get_balanced_Average_examples(input_list):
    # Parse the input list into a dictionary with senseid as key
    senseid_dict = defaultdict(list)
    
    for item in input_list:
        try:
            example, senseid = item.rsplit(' . ', 1)
            senseid_dict[senseid.strip()].append(item.strip())  # Store the full item
        except ValueError:
            continue
        
    if not senseid_dict:
        return []

    # Calculate the average number of examples
    sense_lengths = [len(examples) for examples in senseid_dict.values()]
    avg_examples = int((min(sense_lengths) + max(sense_lengths)) / 2)

    # Randomly sample from each senseid
    balanced_list = []
    for senseid, examples in senseid_dict.items():
        # Ensure the sample size is not larger than the number of available examples
        sample_size = min(avg_examples, len(examples))
        balanced_list.extend(random.sample(examples, sample_size))
    
    return balanced_list

In [37]:
def sense_Tag_Return_pipeline(sentence,postag,wordwsd):
    
    meanings = retrieve_meanings(wordwsd, list_of_lists)

    cleaned_sentence= extract_word_and_index(sentence) 
    #retrieving the data form the dictionary
    filtered_definitions = {key: value for key, value in meanings.items() if postag in key}
    
    if wordwsd in loaded_dict:
        #print("examples found")
        examples=loaded_dict[wordwsd]
        try:
            examples=examples[get_pos(postag)]
            examples = get_balanced_Average_examples(examples)
        except:
            examples=examples
        

    elif word_base(wordwsd) in loaded_dict:
        #print("examples found")
        examples=loaded_dict[word_base(wordwsd)]
        try:
            examples=examples[get_pos(postag)]
            examples = get_balanced_Average_examples(examples)
        except:
            examples=examples
    else:
        examples=None
        

    count=0 #variable to count the tokens 
    
    #prompt=f"Examine the sentence. {instance_meaning}.Return most suitable sense id associated with from below. it contain sense id and it's definition {meanings}. utilize the below examples also to finalize the answer {examples}"
    prompt = f'''You are going to identify the corresponding sense tag of an ambiguous word in English sentences. Use multiple reasoning strategies to increase confidence in your answer.
1. The word "{wordwsd}" has different meanings. Below are possible meanings. Comprehend the sense tags and meanings. Synonyms are provided if available. {filtered_definitions}
2. You can learn more on the usage of each word and the its sense through the examples below. Each sentence is followed by its corresponding sense id. "{examples}"
3. Now carefully examine the sentence below. The ambiguous word is enclosed within <WSD>."{sentence}"
4. Analyze the sentence using the following techniques and identify the meaning of the ambiguous word.
   Focus on keywords in the sentence surrounding the ambiguous word. 
   Think about the overall topic and intent of the sentence. Decide on the sense of the word that makes the most logical sense within the context. 
5. Based on the identified meaning, try to find the most appropriate senseIDs from the below sense tag list. You are given definition of each sense tag too."{filtered_definitions}".
6. If you have more than one senseIDs identified after above steps, you can return the senseIDs in order of confident level, follow the given format to return the value .
7.Return only the finalized senseIDs. Do not add extra detials and explanation. Only senseIDS expected.

if only one senseid is identified then <senseId>
if not <senseId1, senseId2,...> '''
    #print(prompt)
    count+=count_tokens(prompt)
    try:
        output=chat_completion(prompt)
        count+=count_tokens(output)
        return output, count  
    
    except:
        prompt = f'''You are going to identify the corresponding sense tag of an ambiguous word in English sentences. Use multiple reasoning strategies to increase confidence in your answer.
1. The word "{wordwsd}" has different meanings. Below are possible meanings. Comprehend the sense tags and meanings. Synonyms are provided if available. {filtered_definitions}
2. Now carefully examine the sentence below. The ambiguous word is enclosed within <WSD>."{sentence}"
3. Analyze the sentence using the following techniques and identify the meaning of the ambiguous word.
   Focus on keywords in the sentence surrounding the ambiguous word. 
   Think about the overall topic and intent of the sentence. Decide on the sense of the word that makes the most logical sense within the context. 
4. Based on the identified meaning, try to find the most appropriate senseIDs from the below sense tag list. You are given definition of each sense tag too."{filtered_definitions}".
5. If you have more than one senseIDs identified after above steps, you can return the senseIDs in order of confident level, follow the given format to return the value .
6.Return only the finalized senseIDs. Do not add extra detials and explanation. Only senseIDS expected.

if only one senseid is identified then <senseId>
if not <senseId1, senseId2,...>'''
        

        output=chat_completion(prompt)
        count+=count_tokens(output)
        return output, count
        


In [ ]:
#Average frequncy Sampling :GPT 4o
import json
import datetime
import csv
import time
#evaluating the results
no_of_instances=0
no_of_correct_instances=0

file=open("test/semeval2013/semeval2013sample.txt","r",encoding="utf8")


totalTokens=0
for i in file:
    try:
        
        lst=i.split("	")
        sentence,senseid=lst[0],lst[1]
        postag=senseid.split(".")[1] #postag 
        wordfromtext=senseid.split(".")[0] #wsdword
        #print(postag)
        output,token_count=sense_Tag_Return_pipeline(sentence,postag,wordfromtext)
        totalTokens+=token_count        
        print(output.replace("\n", ""))
        no_of_instances+=1
        if senseid in output:
            no_of_correct_instances+=1

        time.sleep(10)

    except Exception as e:
            print(f"An error occurred: {e}")
    
    

file.close()
print(totalTokens)

In [ ]:
#low frequncy Sampling :GPT 4o
import json
import datetime
import csv
import time
#evaluating the results
no_of_instances=0
no_of_correct_instances=0

file=open("test/semeval2013/semeval2013sample.txt","r",encoding="utf8")


totalTokens=0
for i in file:
    try:
        
        lst=i.split("	")
        sentence,senseid=lst[0],lst[1]
        postag=senseid.split(".")[1] #postag 
        wordfromtext=senseid.split(".")[0] #wsdword
        #print(postag)
        output,token_count=sense_Tag_Return_pipeline(sentence,postag,wordfromtext)
        totalTokens+=token_count        
        print(output.replace("\n", ""))
        no_of_instances+=1
        if senseid in output:
            no_of_correct_instances+=1

        time.sleep(10)

    except Exception as e:
            print(f"An error occurred: {e}")
    
    

file.close()
print(totalTokens)


In [ ]:
#highest frequncy Sampling :GPT 4o
import json
import datetime
import csv
import time
#evaluating the results
no_of_instances=0
no_of_correct_instances=0

file=open("test/semeval2013/semeval2013sample.txt","r",encoding="utf8")


totalTokens=0
for i in file:
    try:
        
        lst=i.split("	")
        sentence,senseid=lst[0],lst[1]
        postag=senseid.split(".")[1] #postag 
        wordfromtext=senseid.split(".")[0] #wsdword
        #print(postag)
        output,token_count=sense_Tag_Return_pipeline(sentence,postag,wordfromtext)
        totalTokens+=token_count        
        print(output.replace("\n", ""))
        no_of_instances+=1
        if senseid in output:
            no_of_correct_instances+=1

        time.sleep(10)

    except Exception as e:
            print(f"An error occurred: {e}")
    
    

file.close()
print(totalTokens)

print(totalTokens)

In [ ]:
#Average frequncy Sampling : LLama
import json
import datetime
import csv
import time
#evaluating the results
no_of_instances=0
no_of_correct_instances=0

file=open("test/semeval2013/semeval2013sample.txt","r",encoding="utf8")


totalTokens=0
for i in file:
    try:
        
        lst=i.split("	")
        sentence,senseid=lst[0],lst[1]
        postag=senseid.split(".")[1] #postag 
        wordfromtext=senseid.split(".")[0] #wsdword
        #print(postag)
        output,token_count=sense_Tag_Return_pipeline(sentence,postag,wordfromtext)
        totalTokens+=token_count        
        print(output.replace("\n", ""))
        no_of_instances+=1
        if senseid in output:
            no_of_correct_instances+=1

        time.sleep(10)

    except Exception as e:
            print(f"An error occurred: {e}")
    
    

file.close()
print(totalTokens)

print(totalTokens)

In [ ]:
#highest frequncy Sampling : Llama
import json
import datetime
import csv
import time
#evaluating the results
no_of_instances=0
no_of_correct_instances=0

file=open("test/semeval2013/semeval2013sample.txt","r",encoding="utf8")


totalTokens=0
for i in file:
    try:
        
        lst=i.split("	")
        sentence,senseid=lst[0],lst[1]
        postag=senseid.split(".")[1] #postag 
        wordfromtext=senseid.split(".")[0] #wsdword
        #print(postag)
        output,token_count=sense_Tag_Return_pipeline(sentence,postag,wordfromtext)
        totalTokens+=token_count        
        print(output.replace("\n", ""))
        no_of_instances+=1
        if senseid in output:
            no_of_correct_instances+=1

        time.sleep(10)

    except Exception as e:
            print(f"An error occurred: {e}")
    
    

file.close()
print(totalTokens)

print(totalTokens)

In [ ]:
#lowest sampling : LlaMa
import json
import datetime
import csv
import time
#evaluating the results
no_of_instances=0
no_of_correct_instances=0

file=open("test/semeval2013/semeval2013sample.txt","r",encoding="utf8")


totalTokens=0
for i in file:
    try:
        
        lst=i.split("	")
        sentence,senseid=lst[0],lst[1]
        postag=senseid.split(".")[1] #postag 
        wordfromtext=senseid.split(".")[0] #wsdword
        #print(postag)
        output,token_count=sense_Tag_Return_pipeline(sentence,postag,wordfromtext)
        totalTokens+=token_count        
        print(output.replace("\n", ""))
        no_of_instances+=1
        if senseid in output:
            no_of_correct_instances+=1

        time.sleep(10)

    except Exception as e:
            print(f"An error occurred: {e}")
    
    

file.close()
print(totalTokens)

print("No of Total instance :",no_of_instances)
print("No of correct instance :",no_of_correct_instances)
print(totalTokens)